# Tarefa 1b: Gerar textos usando um prompt que inclui contexto

Neste caderno, você aprenderá a gerar e-mails de resposta destinados a clientes insatisfeitos com a qualidade do atendimento ao cliente prestado pela equipe de suporte. Para dar mais contexto ao modelo, é preciso inserir o conteúdo do e-mail real enviado pelo cliente insatisfeito.

Você adicionará mais complexidade com a ajuda de PromptTemplates para usar o framework LangChain em um caso de uso semelhante. Os PromptTemplates permitem criar shells genéricos que podem ser preenchidos posteriormente com informações e obter saídas de modelo com base em diferentes cenários.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) é um framework para desenvolver aplicações alimentadas por modelos de linguagem. Os principais aspectos deste framework permitem aumentar os grandes modelos de linguagem ao encadear vários componentes para criar casos de uso avançados.

Com o contexto adicional no prompt, o conteúdo produzido neste caderno tem uma qualidade muito melhor e maior relevância do que o conteúdo produzido anteriormente por meio de prompts zero-shot. O prompt usado neste caderno cria um modelo de prompt personalizado do LangChain para adicionar contexto à solicitação de geração de texto.

#### Cenário
Você é Bob, um gerente de atendimento ao cliente na AnyCompany e alguns de seus clientes não estão satisfeitos com o atendimento ao cliente e estão enviando comentários negativos sobre o serviço prestado pelos engenheiros de atendimento ao cliente. Agora, você gostaria de pedir suas sinceras desculpas a esses clientes pelo serviço ruim e recuperar a confiança deles. Você precisa da ajuda de um grande modelo de linguagem (LLM) para gerar vários e-mails de resposta otimistas, personalizados conforme o sentimento descrito no e-mail anterior do cliente.

Nesse cenário, você pode usar o poder dos PromptTemplates do LangChain para criar um shell genérico a fim de gerar respostas de e-mail personalizadas com base no e-mail anterior do cliente. O PromptTemplate incorporará o conteúdo original do e-mail do cliente, permitindo que o LLM entenda o contexto e o sentimento e, em seguida, gere uma resposta relevante e personalizada.

## Tarefa 1b.1: Configurar o ambiente

Nesta tarefa, você vai configurar o ambiente.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tarefa 1b.2: Invocar o modelo de LLM do Bedrock

Nesta tarefa, você cria uma instância da classe do Bedrock com base em grandes modelos de linguagem. Para tal, espera-se um `model_id` que seja o Amazon Resource Name (ARN) do modelo disponível no Amazon Bedrock.

Como opção, você pode transmitir um cliente boto3 criado anteriormente, bem como alguns `model_kwargs` que podem conter parâmetros como `temperature`, `top_p`, `max_token_count` ou `stop_sequences` (mais informações sobre parâmetros podem ser encontradas no console do Amazon Bedrock).

Confira [documentação](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) para ver os IDs dos modelos de geração de texto disponíveis no Amazon Bedrock.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação:** os diferentes modelos são compatíveis com diferentes `model_kwargs`.

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## Tarefa 1b.3: Criar um modelo de prompt personalizado do LangChain

Nesta tarefa, você criará um modelo para o prompt que transmitirá diferentes variáveis de comando a cada processamento. Isso é útil quando você precisa gerar conteúdo com diferentes variáveis de comando que possa vir a obter de um banco de dados.

Na tarefa anterior, fizemos a codificação rígida do prompt. É possível que vários clientes insatisfeitos estejam reclamando sobre a mesma coisa. Nesse caso, você precisa responder aos e-mails dos clientes com um pedido de desculpas, bem como personalizar minimamente a resposta. Na célula a seguir, você verá como criar um `PromptTemplate` para alcançar esse padrão.

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação:** você pode ignorar os avisos com segurança e prosseguir para a próxima célula.

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

Você aprendeu com êxito que invocar o LLM sem qualquer contexto pode não produzir os resultados desejados. Ao adicionar contexto e usar ainda mais o modelo de prompt para restringir a saída do LLM, você conseguiu obter com êxito a saída desejada.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche o arquivo do caderno.
- Retorne à sessão do laboratório e continue com a **Tarefa 2**.